In [1]:
import os

In [26]:
%pwd

'd:\\Project\\Machine Learning\\Text Summarization\\research'

In [2]:
os.chdir("../")

In [28]:
%pwd

'd:\\Project\\Machine Learning\\Text Summarization'

ENTITY

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

CONFIGURATION MANAGER

In [3]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_dirs

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = Config_File_Path,
        params_filepath = Params_File_Path):
        print(config_filepath)
        print(params_filepath)
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_dirs([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_dirs([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir= config.root_dir,
            source_url= config.source_url,
            local_data_file= config.local_data_file,
            unzip_dir= config.unzip_dir
        )
        return data_ingestion_config

In [5]:
ConfigurationManager()

D:\Project\Machine Learning\Text Summarization\config\config.yaml
D:\Project\Machine Learning\Text Summarization\params.yaml
[2024-07-08 13:50:44,069: INFO: common: yaml file: D:\Project\Machine Learning\Text Summarization\config\config.yaml Load Successfully]
[2024-07-08 13:50:44,071: INFO: common: yaml file: D:\Project\Machine Learning\Text Summarization\params.yaml Load Successfully]
[2024-07-08 13:50:44,071: INFO: common: Create direectory at artifacts]


COMPONENTS

In [4]:
import os
import urllib.request as request
import zipfile
from textSummarizer.Logging import logger
from textSummarizer.utils.common import get_size

In [5]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def dowload_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,\
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download with following info \n {headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        zip_file_path: str
        extract the zip file into dir
        Function return None
        """

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)

PIPELINE

In [7]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.dowload_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

D:\Project\Machine Learning\Text Summarization\config\config.yaml
D:\Project\Machine Learning\Text Summarization\params.yaml
[2024-07-08 14:00:21,411: INFO: common: yaml file: D:\Project\Machine Learning\Text Summarization\config\config.yaml Load Successfully]
[2024-07-08 14:00:21,412: INFO: common: yaml file: D:\Project\Machine Learning\Text Summarization\params.yaml Load Successfully]
[2024-07-08 14:00:21,414: INFO: common: Create direectory at artifacts]
[2024-07-08 14:00:21,415: INFO: common: Create direectory at artifacts/data_ingestion]
[2024-07-08 14:00:29,225: INFO: 4263559659: artifacts/data_ingestion/data.zip download with following info 
 Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Option